In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [ ]:
def create_variables():
    rms = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'rms')
    zcr = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'zcr')
    sound = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'sound')
    rms['low'] = fuzz.trimf(rms.universe, [0, 0, 0.5])
    rms['medium'] = fuzz.trimf(rms.universe, [0.2, 0.5, 0.8])
    rms['high'] = fuzz.trimf(rms.universe, [0.5, 1.0, 1.0])
    zcr['low'] = fuzz.trimf(zcr.universe, [0, 0, 0.5])
    zcr['medium'] = fuzz.trimf(zcr.universe, [0.2, 0.5, 0.8])
    zcr['high'] = fuzz.trimf(zcr.universe, [0.5, 1.0, 1.0])
    sound['background'] = fuzz.trimf(sound.universe, [0, 0, 0.5])
    sound['loud'] = fuzz.trimf(sound.universe, [0.5, 1.0, 1.0])
    return rms, zcr, sound

def create_rules(rms, zcr, sound):
    rules = [
        ctrl.Rule(rms['low'] & zcr['low'], sound['background']),
        ctrl.Rule(rms['low'] & zcr['medium'], sound['background']),
        ctrl.Rule(rms['low'] & zcr['high'], sound['background']),
        ctrl.Rule(rms['medium'] & zcr['low'], sound['background']),
        ctrl.Rule(rms['medium'] & zcr['medium'], sound['background']),
        ctrl.Rule(rms['medium'] & zcr['high'], sound['loud']),
        ctrl.Rule(rms['high'] & zcr['low'], sound['loud']),
        ctrl.Rule(rms['high'] & zcr['medium'], sound['loud']),
        ctrl.Rule(rms['high'] & zcr['high'], sound['loud']),
    ]
    return rules

In [ ]:
def create_system():
    rms, zcr, sound = create_variables()
    rules = create_rules(rms, zcr, sound)
    system = ctrl.ControlSystem(rules)
    sim = ctrl.ControlSystemSimulation(system)
    return sim

In [ ]:
def classify_sound(sim, rms_val, zcr_val):
    sim.input['rms'] = rms_val
    sim.input['zcr'] = zcr_val
    sim.compute()
    output = sim.output['sound']
    return 'loud sound' if output >= 0.5 else 'background sound'

In [ ]:
sim = create_system()
rms_val = 0.7
zcr_val = 0.3
result = classify_sound(sim, rms_val, zcr_val)
result